In [12]:
import re

haasecits = {}
haasechaps = {}

newchaps = {}
newcits = {}

smithcits = {}
smithchaps = {}

dalechaps = {}
dalenotes = {}

smithnotes = {}

betantcits = {}
betantchaps = {}

reflist = {}
footnotelist = {}
notetexts = {}
sawnotes= {}

nosects = 1

def additem(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1

def getcits(dir,urn,citlist,chaplist):
    cursection = ''
    curchapter = ''
    curbook = ''
    global dalecits
    global reflist
    global footnotelist
    global notetexts
    global sawnotes
    innote = 0
    curpage = ''
    fname = dir + urn + '-nobase.xml'
    
    f = open(fname)
    for l in f:
        
        m = re.search('<pb n="([^"]+)"',l)
        if(m):
            curpage = m[1]
            curnote = 0
            curref = 0
    
        refs = re.findall('<hi rend="superscript">([0-9]+)</hi>',l)
        for foo in refs:
            curref = curref + 1
            #if(not re.search('^[0-9]+$',foo)):
            #    print('badtref',curpage,foo,curref)
            currefmark = curpage + '.n.' + str(curref)
            additem(currefmark,reflist)
            
            
        
        
        
        if(re.search('^<note[^>]+type="footnote">([^\s]+)\s+(.+)</note>',l)):
            m = re.search('^<note[^>]+type="footnote">([^\s]+)\s+(.+)</note>',l)
            curnote = curnote + 1
            curnotetext = m[2]
            #if(not re.search('^[0-9]+',m[1])):
                #print('badref',curpage,curnote,m[1])
            #    curnoteref =  curpage + '.n.' + str(curnote)
            #else:
            #    curnoteref =  curpage + '.n.' + int(m[1])
            curnoteref =  curpage + '.n.' + str(curnote)
            additem(curnoteref,footnotelist)
            #print(curnoteref)
            notetexts[curnoteref] = curnotetext

    
    f.close()
    
    for foo in reflist:
        if(not foo in notetexts):
            print('no note',foo)
    
    for foo in notetexts:
        if(not foo in reflist):
            print('no ref',foo)
    
    f = open(fname)
    text = f.read()
    workfname = '/tmp/' + urn + '.xml'
    workf = open(workfname,'w')
    
        
    while(re.search('(<note[^>]*>)(.+?</note>)',text)):
        m = re.search('(<note[^>]*>)(.+?</note>)',text)
        noteref = m[1]
        notetext = m[2]
        if(not re.search('xml:lang',noteref)):
            
            if(re.search('lang="grc"',notetext)):
                text = re.sub('(<note)([^>]*>.+?</note>)','\g<1> xml:lang="mul"\g<2>',text,1)
            else:
                text = re.sub('(<note)([^>]*>.+?</note>)','\g<1> xml:lang="eng"\g<2>',text,1)
            
                
        
        text = re.sub('(<)(note[^>]*>.+?</)(note>)','\g<1>Z\g<2>Z\g<3>',text,1)
    
    text = re.sub('Znote','note',text)
    
    text = re.sub('(<note)([^:>]*>)','\g<1> xml:lang="mul"\g<2>',text)
    
    
    
    print(text,file=workf)
    #print(text)
    
    f.close()
    workf.close()
    
    f = open(workfname)
    outfname = '/Users/gcrane/github/first1kwork/data/tlg0003/tlg001/' + urn + '.xml'
    outf = open(outfname,'w')
    
    for l in f:
        l = re.sub('\s+$','',l)
        
        m = re.search('<pb n="([^"]+)"',l)
        if(m):
            curpage = m[1]
            curnote = 0
            curref = 0
            
        while(re.search('<hi rend="superscript">([0-9]+)</hi>',l)):
            m = re.search('<hi rend="superscript">([0-9]+)</hi>',l)
            curref= curref + 1
            curkey = curpage + '.n.' + str(curref)
            l = re.sub('<hi rend="superscript">([0-9]+)</hi>','<note xml:lang="mul">'+ notetexts[curkey]+'</note>',l,1)
    
    
        
        m = re.search('subtype="book"\s*n="([^"]+)"',l)
        if(m):
            curbook = m[1]
            baseurn = 'cts:urn:' + urn
            if(not re.search('xml:base',l)):
                l = re.sub('(subtype="[^"]+")','\g<1> xml:base="'+baseurn+'"',l)
            
            
        m = re.search('subtype="chapter"\s*n="([^"]+)"',l)
        if(m):
            curchapter = m[1]
            baseurn = 'cts:urn:' + urn + ':' + curbook
            if(not re.search('xml:base',l)):
                l = re.sub('(subtype="[^"]+")','\g<1> xml:base="'+baseurn+'"',l)
            additem(curbook + '.' + curchapter,chaplist)
            
        m = re.search('subtype="section"\s*n="([^"]+)"',l)
        if(m):
            cursection = m[1]
            curcit = curbook + '.' + curchapter + '.' + cursection
            additem(curcit,citlist)
            baseurn = 'cts:urn:' + urn + ':' + curbook + '.' + curchapter
            if(not re.search('xml:base',l)):
                l = re.sub('(subtype="[^"]+")','\g<1> xml:base="'+baseurn+'"',l)
            
        if(re.search('<note',l)):
            innote = 1
            
        if(re.search('</note>',l)):
            innote = 0
        print(l,file=outf)
    
    f.close()
    
newfname = 'tlg0003.tlg001.1st1K-ita1'

getcits('florence/',newfname,haasecits,haasechaps)

smithfname = 'tlg0003.tlg001.1st1K-eng1'

getcits('smith/',smithfname,smithcits,smithchaps)

#betantfname = 'tlg0003.tlg001.1st1K-fre1'

#getcits('betant/',betantfname,betantcits,betantchaps)



for foo in haasechaps:
    continue
    if(not foo in smithchaps):
        print('haase chap',foo)

for foo in smithchaps:
    continue
    if(not foo in haasechaps):
        print('haase chap',foo)

for foo in haasecits:
    continue
    if(haasecits[foo]> 1):
        print('haase dupes',foo)
    if(not foo in smithcits):
        print('smith sect missing',foo)


for foo in smithcits:
    continue
    if(not foo in haasecits):
        print('haasecits sect missing',foo)

       

In [8]:
print(len(notetexts),len(reflist))
for foo in notetexts:
    print(foo,notetexts[foo])

0 0


In [21]:
haasecits


{'1.80.1': 1,
 '1.81.1': 1,
 '1.82.1': 1,
 '1.83.1': 1,
 '1.84.1': 1,
 '1.85.1': 1,
 '2.35.1': 1,
 '2.35.2': 1,
 '2.35.3': 1,
 '2.36.1': 1,
 '2.36.2': 1,
 '2.36.3': 1,
 '2.36.4': 1,
 '2.37.1': 1,
 '2.37.2': 1,
 '2.37.3': 1,
 '2.38.1': 1,
 '2.38.2': 1,
 '2.39.1': 1,
 '2.39.2': 1,
 '2.39.3': 1,
 '2.39.4': 1,
 '2.40.1': 1,
 '2.40.2': 1,
 '2.40.3': 1,
 '2.40.4': 1,
 '2.40.5': 1,
 '2.41.1': 1,
 '2.41.2': 1,
 '2.41.3': 1,
 '2.41.4': 1,
 '2.41.5': 1,
 '2.42.1': 1,
 '2.42.2': 1,
 '2.42.3': 1,
 '2.42.4': 1,
 '2.43.1': 1,
 '2.43.2': 1,
 '2.43.3': 1,
 '2.43.4': 1,
 '2.43.5': 1,
 '2.43.6': 1,
 '2.44.1': 1,
 '2.44.2': 1,
 '2.44.3': 1,
 '2.44.4': 1,
 '2.45.1': 1,
 '2.45.2': 1,
 '2.46.1': 1,
 '2.46.2': 1,
 '4.59.1': 1,
 '4.60.1': 1,
 '4.61.1': 1,
 '4.62.1': 1,
 '4.63.1': 1,
 '4.64.1': 1,
 '4.85.1': 1,
 '4.86.1': 1,
 '4.87.1': 1}

In [22]:
import re
fname = 'gursching/tlg0003.tlg001.1st1K-ger1-nobase.xml'

curdiv = 0
f = open(fname)
showlines = 0
for l in f:
    l = re.sub('\s+$','',l)
    if(re.search('<div',l)):
        curdiv = curdiv + 1
        if(re.search('chapter',l) and curdiv != 3):
            print(curdiv,l)
        if(re.search('section',l) and curdiv != 4):
            print(curdiv,l)
            showlines = 1
    if(re.search('</div>',l)):
        curdiv = curdiv - 1
        #print(curdiv,l)
    if(showlines):
        print(l)


f.close()